In [47]:
#Sort Different columns: 
import pandas as pd 
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit_aer.primitives import Sampler
import re
import openpyxl

df = pd.read_excel('Files/Student-1.xlsx')
type = detectColumns(df,df.columns)
print(type)    

{'id': {'0': 89, '1': 10}, 'name': {'1': 60, '0': 39}, 'properties': {'0': 39, '1': 60}}


In [18]:
import pandas as pd
import numpy as np
def digitSorting(df,col,asc):
    df[col] = df[col].astype(str)
    df = df.sort_values(by=col,ascending=asc, ignore_index=True)
    return df

df = pd.read_csv("Files/student_dataset.csv")
df = digitSorting(df,'Grade',asc=False)
print(df)


         Student_Names   Phone_No.  Math  Physics  Chemistry Grade  \
0      Karina Williams  9092378235    23       26         36     F   
1       Kristin Parker  9226776504    12       25         38     F   
2           Amber Leon  9932583197    38       23         17     F   
3         Jodi Kennedy  9443258069    17       26         45     F   
4     Natasha Stephens  9828839720    29       32         11     F   
...                ...         ...   ...      ...        ...   ...   
8995     Edward Palmer  9817654825    98       81         84     A   
8996      Tracy Miller  9176542338    76      100         89     A   
8997      Nichole Snow  9108855306    93       62         98     A   
8998      Peter Maddox  9967091167    71       89         90     A   
8999     Marissa Smith  9601081991    96       51        100     A   

                   Comment  Roll No.           School Name  \
0                   Failed    536668  Martin Luther School   
1                   Failed    55019

In [81]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def sortRollCol(df, col,asc):
    digits_df = df[col].astype(str).apply(lambda x: pd.Series(list(x)))
    
    digits_df['original_index'] = df.index
    
    sorted_digits_df = recursiveSort(digits_df)

    sorted_indices = sorted_digits_df['original_index'].values
    if asc == False:
        sorted_indices = reversed(sorted_indices)
        
    sorted_df = df.loc[sorted_indices].reset_index(drop=True)

    return sorted_df


def recursiveSort(df_digits, col=0):
    if col >= int(len(df_digits.columns) - 1):  # exclude 'original_index' column
        return df_digits

    # Sort by the current digit column
    df_digits = df_digits.sort_values(by=col, kind='stable', ignore_index=True)

    # Group by current digit and recursively sort each group
    result = []
    for value, group in df_digits.groupby(col, sort=False):
        sorted_group = recursiveSort(group.reset_index(drop=True), col + 1)
        result.append(sorted_group)
    
    return pd.concat(result, ignore_index=True)

In [ ]:
import re
def dataClean(df,colTypes):
    for i in df.index[(df.isna().sum(axis=1)==len(df.columns))]:
        df.drop(index=i, inplace=True)
        
    for i in colTypes.keys():
        if df[i].isna().sum() >=len(df[i])-2:
            df.drop(columns=[i],inplace=True)
            continue
        
        if df[i].isna().sum() != 0:#IF column has no type
                print(i)
                if (colTypes[i]['0'] == 60 or colTypes[i]['1'] == 40): # if column is type of year only
                    df[i] = df[i].fillna(2099)
                elif colTypes[i]['0'] == 30 or  colTypes[i]['1'] == 70:#if column is type of date only 
                    df[i] = df[i].fillna(pd.to_datetime('2030-01-01'))
                elif colTypes[i]['0'] == 20 or  colTypes[i]['1'] == 80 :#if column is type of date and time 
                    df[i]= df[i].fillna('2030-01-01')
                elif colTypes[i]['0'] == 70 or colTypes[i]['1']==30: #Roll no type
                    pass
                elif colTypes[i]['1']==50 or colTypes[i]['0']==50:#if it is of type id 
                    df[i]=df[i].fillna("NAN-123-00")
                elif colTypes[i]['0']==90 or colTypes[i]['1'] ==10:#if it of type oneor2digit
                    df[i] = df[i].fillna(0)
                elif colTypes[i]['0'] == 80 or  colTypes[i]['1'] == 20 : #if it type of nemric
                    df[i]=df[i].fillna(0)
                elif colTypes[i]['0'] == 40 or  colTypes[i]['1'] == 60 : #string or object
                    df[i] = df[i].fillna('NULL-NAN')
                    
        elif len(set(df[i]))==1:
            for k in range(2):
                df[i].at[k]="Changed For better Clustering"
        continue
    return df

df = pd.read_csv('Files/business-financial-data-december-2024-quarter-csv.csv')

col = detectColumns(df,df.columns)
print(df)
df = dataClean(df,col)
print(df)


In [ ]:
def findDuplicate(series):
    series = series[series.duplicated(keep=False)]
    half = set(series)
    if len(half)>=len( series)*0.30:
        return 9
    return 11

df = pd.read_csv('Files/employee_time_log.csv')
dup = findDuplicate(df['Employee_ID'])
print(np.array(dup))


9


In [48]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd

def cluster_text_column(df: pd.DataFrame, column_name: str, k_min: int = 2, k_max: int = 40, plot: bool = True) -> pd.DataFrame:
    comments = df[column_name].dropna().astype(str).tolist()

    # Sentence embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')
    X = model.encode(comments)

    # Find the best number of clusters using silhouette scores
    k_range1 = list(range(k_min, 11,2))
    k_range2 = list(range(12,25,3))
    k_range3 = list(range(28,40,4))
    k_range= k_range1+k_range1+k_range2+k_range3
    scores = silhouetteScores(k_range,X)
    print(scores)
    best_k_range =dict(sorted(scores.items(), key=lambda x: x[1], reverse=True)[:2])
    print(best_k_range)
    
    k_scores = {}
    for i in best_k_range.keys():
        best_k = silhouetteScores(list(range(int(i) - 1, int(i) + 2)), X)
        max_k = max(best_k, key=best_k.get)
        k_scores[int(max_k)] = best_k[max_k]
        if best_k[max_k] == 1.0:
            break

    final_k = max(k_scores, key=k_scores.get)
    best_score = k_scores[final_k]

    print(f"\nBest K: {final_k}, Highest Silhouette Score: {best_score:.4f}")

    final_model = KMeans(n_clusters=final_k, random_state=42, n_init=20, max_iter=550)
    final_labels = final_model.fit_predict(X)

    df_clustered = df.copy()
    df_clustered["cluster_label"] = -1
    df_clustered.loc[df[column_name].notna(), "cluster_label"] = final_labels
    df_clustered = df_clustered.sort_values(by='cluster_label',ignore_index=True)
    
    return df_clustered

def silhouetteScores(k_range, X):
    if 1 in k_range:
        k_range.remove(1)
    scores = {}
    a = 0
    index = 0
    while index < len(k_range):
        k = k_range[index]

        kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        scores[k] = score
        print(f"K={k}, Silhouette Score={score:.4f}")

        # Check conditions and expand k_range dynamically
        if len(scores.values()) >= 14 and all(x < 0.7 for x in scores.values()):
            print("nye")
            k_range = list(range(k + 6, 170, 6))
            continue
        
        if len(scores.values()) >= 10 and all(x < 0.48 for x in scores.values()):
            print("kye")
            k_range = list(range(k + 6, 112, 5))
            continue
        
        if len(scores.values()) >= 7 and all(x < 0.32 for x in scores.values()):
            print("bye")
            k_range = list(range(k + 5, 90, 4))
            continue
        
        if len(scores.values()) >= 2:
            if all(x < 0.25 for x in scores.values()):
                print("hii")
                k_range = list(range(k + 4, 60, 3))
                continue
            values = list(scores.values())
            if values[a] > values[a + 1] or len(set(values[-2:])) == 1:
                return scores
            
            a += 1
        index += 1
    
    return scores

df = pd.read_csv('Files/customers-1000.csv') 
df = cluster_text_column(df,'Country')
print(df)

K=2, Silhouette Score=0.0900
K=4, Silhouette Score=0.0630
hii
K=11, Silhouette Score=0.0812
hii
K=18, Silhouette Score=0.1076
hii
K=25, Silhouette Score=0.1375
hii
K=32, Silhouette Score=0.1758
hii
K=39, Silhouette Score=0.2147
bye
K=48, Silhouette Score=0.2659
bye
K=57, Silhouette Score=0.2952
bye
K=66, Silhouette Score=0.3345
kye
K=77, Silhouette Score=0.3901
kye
K=88, Silhouette Score=0.4455
kye
K=99, Silhouette Score=0.4965
{2: np.float32(0.09003331), 4: np.float32(0.06297199), 11: np.float32(0.08122044), 18: np.float32(0.10762339), 25: np.float32(0.13751623), 32: np.float32(0.17578171), 39: np.float32(0.21473537), 48: np.float32(0.26586828), 57: np.float32(0.29524392), 66: np.float32(0.33447242), 77: np.float32(0.39007452), 88: np.float32(0.44553006), 99: np.float32(0.4964698)}
{99: np.float32(0.4964698), 88: np.float32(0.44553006)}
K=98, Silhouette Score=0.4928
K=99, Silhouette Score=0.4965
K=100, Silhouette Score=0.5020
K=87, Silhouette Score=0.4393
K=88, Silhouette Score=0.4455

In [19]:
    
df = pd.read_excel('Files/All.xlsx') 
asc=True
df= digitSorting(df,col='STATUS',asc=asc)
type={'1':10,'0':9}
df = multiIndex(dataFrame=df,colToCheck='STATUS',colType=type,asc=asc)  
df

Can only use .dt accessor with datetimelike values
last index called
[368, 632]
['Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of F'
 'Group of F' 'Group of F' 'Group of F' 'Group of F' 'Group of

C:\Users\tikes\AppData\Local\Temp\ipykernel_20228\1671503201.py:92: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()


STATUS Employee_ID Start_Date      Student_Names  Roll No.  \
Group      Sr No.                                                              
Group of F 0           F       E7771 2024-12-09   Donald Contreras    524613   
           1           F       E6811 2025-01-23    Richard Farrell    566434   
           2           F       E4669 2024-12-28    William Preston    578153   
           3           F       E9357 2024-12-17      Anthony Smith    557050   
           4           F       E2090 2024-12-26       Daniel Jones    598470   
...                  ...         ...        ...                ...       ...   
Group of R 995         R       E6059 2024-12-24      John Bradford    584581   
           996         R       E8883 2025-01-16  Christopher Brown    575991   
           997         R       E4518 2024-12-18      Adrian Taylor    537931   
           998         R       E9541 2024-12-22     Jessica Garcia    555035   
           999         R       E9115 2025-01-30         Mary Young    528216   

                  Grade      f2  year  
Group      Sr No.                      
Group of F 0         B+ -0.0930  2017  
           1          A  0.6412  2011  
           2          D  0.7002  2011  
           3          C  0.4346  2011  
           4         B+  0.4112  2011  
...                 ...     ...   ...  
Group of R 995        A  0.8152  2011  
           996        B  0.9424  2011  
           997        C  0.9920  2011  
           998        D  0.5414  2011  
           999        B  0.3182  2014  

[1000 rows x 8 columns]

In [ ]:
def multiIndex(dataFrame, colToCheck, colType, yearOnly = False, asc=True):
    # Step 1: Get last indices of each years
    skip= False
    if yearOnly == True:
        yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck], True)
    else: 
        if (colType['0'] == 30 or  colType['1'] == 70) or (colType['0'] == 20 or  colType['1'] == 80):
            yearsWithLastIndex = get_last_indices_of_each_year(pd.to_datetime(dataFrame[colToCheck]),False)
        elif(colType['1']==30 or colType['0']==70) or (colType['1'] ==50 or colType['0']==50):
            yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck],False,True,asc=asc)
            asc=True
        elif(colType['1']==60 or colType['0']==40):
            skip = True
        else:   
            yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck], False)
        if asc==False and skip !=True:
            yearsWithLastIndex = dict(reversed(list(yearsWithLastIndex.items())))
            
    if skip ==False:   
        
        nameOfGroups = list(yearsWithLastIndex.keys())
        last_indices = list(yearsWithLastIndex.values())

        # Step 2: Compute counts from last indices
        group_sizes = []
        prev = -1
        for idx in last_indices:
            group_sizes.append(idx - prev)
            prev = idx
        print(group_sizes)
        # Step 3: Create array per group
        objOfGroups = {
            f'key{i}': np.array([f'Group of {year}'] * group_sizes[i]) for i, year in enumerate(nameOfGroups)
        }
    
        # Step 4: Combine into one array
        outside = np.concatenate(list(objOfGroups.values()))
    else:
        outside = np.array(df['cluster_label'])
    
    inside = np.arange(len(outside))

    print(outside)
    # Step 5: Create inside index
    
    multi_index = pd.MultiIndex.from_arrays([outside, inside], names=["Group", "Sr No."])
    
    dataFrame = dataFrame.reindex(range(len(multi_index)))
    dataFrame.set_index(multi_index,inplace=True,)
    print("Multicalled")
    return dataFrame

def get_last_indices_of_each_year(date_series, YearOnly=False,rol=False, a=0.60,asc=True):
    
    # data_series = data_series.apply(pd.to_numeric, errors='coerce').astype('Int64')
    if rol==True and findDuplicate(date_series).count() <=10:
            df = pd.DataFrame({'year':date_series},index=np.arange(len(date_series))).astype(str)
            df['half'] = df['year'].apply(lambda x: x[:round(len(str(x))*a)])
            print(a)
            last_indices =  df.groupby('half',).apply(lambda x: x.index[-1]).to_dict()
            if asc ==False:
                last_indices = dict(reversed(list(last_indices.items())))
                
            group_sizes = []
            prev = -1
            k=0
            for i,idx in enumerate(last_indices.values()):
                group_sizes.append(idx - prev)
                prev = idx
                if group_sizes[i]<= 15:
                    k=k+1
            print(k)
            if round(len(last_indices)*0.45)<=k and a>=0.20:
                last_indices = get_last_indices_of_each_year(date_series,False,True,a=round(a-0.10,3), asc=asc)
            return last_indices
    if YearOnly == True:
        print("yearOnly work")
        df = pd.DataFrame({'year': date_series}, index=np.arange(len(date_series)))
    
    else:
        # Extract year
        try:
            years = date_series.dt.year
            # Create a DataFrame with index
            df = pd.DataFrame({'year': years}, index=np.arange(len(date_series)))
            print("Year Extracted")
        except Exception as e:
            #Create a DataFrame with index
            print(e)
            df = pd.DataFrame({'year': date_series}, index=np.arange(len(date_series)))
        
    # Get last index of each year group
    last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()
    print("last index called")
    return last_indices


In [108]:
a = ['e,','e','t','t']
b = np.array([1,2,3,4])
m = pd.MultiIndex.from_arrays([a,b])
m

MultiIndex([('e,', 1),
            ( 'e', 2),
            ( 't', 3),
            ( 't', 4)],
           )

In [75]:
# Example usage
import pandas as pd
df = pd.read_csv('Files/201516ListofMonasticSchools_1_0.csv')  # Your dataset with 'Roll No.'
asc = True
df = sortRollCol(df,'U-DISE Code',asc=asc)

print(df)
type = {
        '0' : 70, 
        '1' : 30
        }
df = multiIndex(df,colToCheck='U-DISE Code',colType=type,asc=asc)
df

    Sl.no.  U-DISE Code Name of Monastic School under SSA
0       39  11010100106                  NAGA MONASTIC PS
1       40  11010100302            SINGCHIT NADAK MON. PS
2       41  11010100602                       SINGHIK MPS
3       38  11010104301                    RINGHIM MON PS
4       31  11010200105     LACHEN NGEDUP CHOLING MON. PS
..     ...          ...                               ...
74      24  11040901404           CHAGSAM MONASTIC SCHOOL
75      15  11041002003              SANG MONASTIC SCHOOL
76      16  11041002004    SANG NORBULING MONASTIC SCHOOL
77      14  11041002102          CHUNKHAR MONASTIC SCHOOL
78      19  11041100105    LINKEY MACHONG MONASTIC SCHOOL

[79 rows x 3 columns]
0.6
24
0.5
24
0.4
1
[18, 13, 20, 28]
['Group of 1101' 'Group of 1101' 'Group of 1101' 'Group of 1101'
 'Group of 1101' 'Group of 1101' 'Group of 1101' 'Group of 1101'
 'Group of 1101' 'Group of 1101' 'Group of 1101' 'Group of 1101'
 'Group of 1101' 'Group of 1101' 'Group of 1101' 

C:\Users\tikes\AppData\Local\Temp\ipykernel_7736\416729770.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_indices =  df.groupby('half',).apply(lambda x: x.index[-1]).to_dict()
C:\Users\tikes\AppData\Local\Temp\ipykernel_7736\416729770.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_indices =  df.groupby('half',).apply(lambda x: x.index[-1]).to_dict()
C:\Users\tikes\AppData\Local\Temp\ip

Sl.no.  U-DISE Code Name of Monastic School under SSA
Group         Sr No.                                                       
Group of 1101 0           39  11010100106                  NAGA MONASTIC PS
              1           40  11010100302            SINGCHIT NADAK MON. PS
              2           41  11010100602                       SINGHIK MPS
              3           38  11010104301                    RINGHIM MON PS
              4           31  11010200105     LACHEN NGEDUP CHOLING MON. PS
...                      ...          ...                               ...
Group of 1104 74          24  11040901404           CHAGSAM MONASTIC SCHOOL
              75          15  11041002003              SANG MONASTIC SCHOOL
              76          16  11041002004    SANG NORBULING MONASTIC SCHOOL
              77          14  11041002102          CHUNKHAR MONASTIC SCHOOL
              78          19  11041100105    LINKEY MACHONG MONASTIC SCHOOL

[79 rows x 3 columns]

In [46]:
from qiskit import QuantumCircuit
import pandas as pd
import math
def detectColumns(df, prioColumns):
    result = {}    
    for col in prioColumns:
        qc =  QuantumCircuit(1,1)
        col_data = df[col]
        col_str = df[col].astype(str).str.strip()
        # 3. Check for Date values (type 2)
        if check_date_format(col_str) :
            p = 0.70000
                
        # 4. Check for DateTime values (type 3)
        elif check_datetime_format(col_str):
            p = 0.800000
            
        elif OneOr2digitDetection(col_data) and detectIdTypeCol(col_data)==False:
            p = 0.100000
        # 1. Check for Roll Numbers (type 4)
        elif pd.api.types.is_numeric_dtype(col_data):
            p = 0.2000
            # 2. Check for Year values (type 1)
            if check_year_values(col_data):
                p = 0.40000
            elif check_roll_number(col_data):
                p = 0.30000

        elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            p = 0.600000
            if detectIdTypeCol(col_data):
                p = 0.5
        
        angle = 2 * math.asin(math.sqrt(p))
        qc.ry(angle, 0)
        # Initialize result as 0 (unrecognized type)
        result[col] = measurCir(qc,0)
        
    return result

def check_roll_number(col_data):
    try:
        # Convert numbers to strings for checking patterns
        sample_start = [str(i) for i in col_data.head(10)]
        sample_middle = [str(i) for i in col_data.iloc[int(len(col_data)/2)-5:int(len(col_data)/2)+5]]
        sample_end = [str(i) for i in col_data.iloc[-10:]]
        
        # Combine samples
        samples = sample_start + sample_middle + sample_end
        
        # Check if all numbers have the same length and >= 5 digits
        if len(set(len(str(x)) for x in samples)) == 1:
            length = len(str(samples[0]))
            if length >= 5:
                # Check if all numbers start with the same digit
                first_digits = str(samples[0])[0]
                return all(str(x).startswith(first_digit) for x in samples)
    except:
        pass
    return False

def check_year_values(col_data):
    if pd.api.types.is_string_dtype(col_data):
        try:
            col_data = pd.to_numeric(col_data)
        except:
            pass
    # Handle if column is numeric and looks like a year
    if pd.api.types.is_numeric_dtype(col_data) or  pd.api.types.is_float_dtype(col_data) or  pd.api.types.is_integer_dtype(col_data):
        if col_data.dropna().empty == False:
            if pd.api.types.is_float_dtype(col_data):
            # Check if float values have only 2 decimal places
                if col_data.dropna().apply(lambda x: round(x, 2) == x).all():
                    if col_data.dropna().between(1800, 2050).all():
                        return True # Only year
            # For integer values
            elif col_data.dropna().between(1800, 2100).all():
                True # Only year
                
    return False

def check_date_format(col_str):
    # Check for common date formats (yyyy-mm-dd, dd-mm-yyyy, etc.)
    date_pattern = r'^\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}$'
    return col_str.str.match(date_pattern).all()

def check_datetime_format(col_str):
    # Check for datetime format (date + time)
    datetime_pattern = r'\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}.*\d{1,2}:\d{2}'
    return col_str.str.contains(datetime_pattern).all()

def detectIdTypeCol(col_data):
    if pd.api.types.is_string_dtype(col_data):
        pattern = r'\b[A-Z0-9]{1,4}[-_./]?[A-Z0-9]{2,6}[-_./]?[A-Z0-9]{0,5}\b'
        return all(re.fullmatch(pattern, item) for item in col_data)
    
def OneOr2digitDetection(col_data):
    non2Digit=0
    for i,v  in enumerate(col_data):
        if len(str(v)) > 2 and str(v)!='nan':
            non2Digit=non2Digit+1
    if len(col_data)/2.2 > non2Digit:
        return True
    return False

def detectSimpleDtypes(col_data):
    if pd.api.types.is_integer_dtype(col_data):
        return 'allInt'
    if pd.api.types.is_float_dtype(col_data):
        if col_data.isna().all()== False:
            return 'numaric'
    if pd.api.types.is_string_dtype(col_data) or pd.api.types.is_object_dtype(col_data):
        return 'str'
    return None

def clusterDateTimeCol(fContent, col,no,ascending=True):
    if no ==1:
        # Try to detect and sort if the column is just year values
        fContent = fContent.sort_values(by=col,ignore_index=True, ascending=ascending)
        fContent = fContent.reset_index(drop=True)
    elif no == 2 or 3:
        # Now, try to detect proper date/datetime columns
        try:
            # Avoid processing numeric-only or zero-filled columns
            sample_vals = fContent[col].astype(str).str.strip().replace('0', np.nan).dropna()
            if len(sample_vals) == 0:
                return fContent# All values are zero or empty-like
            
            # Try parsing
            try:
                parsed_col = pd.to_datetime(fContent[col], errors='raise')
                print('try')
            except Exception as e:
                parsed_col = pd.to_datetime(fContent[col], errors='raise',dayfirst=True)
                print(e)
               
            if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()) and no == 2:  # Only date
                fContent[col] = parsed_col
                fContent = clean_and_sort_date_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
                # Replace original column with parsed datetime values
            elif no == 3:  # Date + time
                fContent[col] = parsed_col
                fContent = handle_datetime_column(fContent, col, ascending)
                fContent = fContent.reset_index(drop=True)
        except Exception as e:
            return fContent # Not a datetime column
        
    return fContent 

def clean_and_sort_date_column(dff, column_name, ascending=True):
        try:
            
            # Step 2: Drop NaT (invalid formats)
            dff = dff.dropna(subset=[column_name])
            
            # Step 3: Sort the DataFrame by that column
            dff = dff.sort_values(by=column_name, ascending=ascending)

            # Optional: Format to clean date string (YYYY-MM-DD)
            dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')
            print(f"{column_name} date called ")
            return dff
        
        except Exception as e:
            print(f"⚠️ Error while processing date column: {e}")
            return dff

def handle_datetime_column(df, column_name, ascending):
    print(f"{column_name} dateTime")
    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([pd.api.is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name,ascending=ascending).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [45]:
def measurCir(qc,q_num):
    qc.measure(q_num,q_num)
    simulator = AerSimulator()
    # Transpile & run
    compiled = transpile(qc, simulator)
    r = simulator.run(compiled, shots=100000).result()
    counts = r.get_counts()
    for k,v in counts.items():
        counts[k] = int(v/1000)
    return counts